## CIFAR-100 under PGD attack  

In [1]:
import timm
import time 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from medmnist import INFO
import numpy as np
import faiss
import copy
from tqdm import tqdm

from torch.nn.functional import softmax, cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import clip
import os 

import warnings
warnings.filterwarnings("ignore")

In [2]:
device_name = 'cuda:0' 

device = torch.device(device_name if torch.cuda.is_available() else "cpu")

In [4]:
### import archetecture of models 

from model_codes_cifar100.vgg_cifar100 import * 
from model_codes_cifar100.resnet_cifar100 import * 
from model_codes_cifar100.densenet_cifar100 import * 
from model_codes_cifar100.xception_cifar100 import * 
from model_codes_cifar100.googlenet_cifar100 import * 
from model_codes_cifar100.inceptionv3_cifar100 import * 
from model_codes_cifar100.mobilenetv2_cifar100 import *  

In [5]:
## getting the archetecture of model from models.vgg_models 
vgg19_model = VGG('VGG19')
resnet50_model = resnet50() 
densenet169_model = densenet169() 
xception_model = xception() 
googlenet_model = googlenet()  
inceptionv3_model = inceptionv3()
mobilenetv2_model = mobilenetv2()

In [6]:
trained_models_directory = "trained_models_cifar100/" 

vgg_19_model_file_name = "vgg19_cifar100.pth" 
resnet50_model_file_name = "resnet50_cifar100.pth" 
densenet169_model_file_name = "densenet169_cifar100.pth" 
xception_model_file_name = "xception_cifar100.pth" 
googlenet_model_file_name = "googlenet_cifar100.pth" 
inception_model_file_name = "inceptionv3_cifar100.pth"
mobilenetv2_model_file_name = "mobilenetv2_cifar100.pth" 

vgg19_path = os.path.join(trained_models_directory, vgg_19_model_file_name) 
resnet50_path = os.path.join(trained_models_directory, resnet50_model_file_name) 
densenet169_path = os.path.join(trained_models_directory, densenet169_model_file_name) 
xception_path = os.path.join(trained_models_directory, xception_model_file_name) 
googlenet_path = os.path.join(trained_models_directory, googlenet_model_file_name) 
inception_path = os.path.join(trained_models_directory, inception_model_file_name)
mobilenetv2_path = os.path.join(trained_models_directory, mobilenetv2_model_file_name)

## load the model 
vgg19_model.load_state_dict(torch.load(vgg19_path, map_location=device_name)['net'])
resnet50_model.load_state_dict(torch.load(resnet50_path, map_location=device_name)['net'])
densenet169_model.load_state_dict(torch.load(densenet169_path, map_location=device_name)['net'])
xception_model.load_state_dict(torch.load(xception_path, map_location=device_name)['net'])
googlenet_model.load_state_dict(torch.load(googlenet_path, map_location=device_name)['net'])
inceptionv3_model.load_state_dict(torch.load(inception_path, map_location=device_name)['net'])
mobilenetv2_model.load_state_dict(torch.load(mobilenetv2_path, map_location=device_name)['net'])

<All keys matched successfully>

In [7]:
trained_pool = [
                vgg19_model.eval(), 
                resnet50_model.eval(), 
                densenet169_model.eval(), 
                # mobilenetv2_model.eval(), 
                googlenet_model.eval(), 
                xception_model.eval(), 
                inceptionv3_model.eval()
               ]


model_names = [
    "VGG19",
    "ResNet50",
    "DenseNet169",
    # "MobileNetV2",
    "GoogleNet", 
    "Xception", 
    "InceptionV3"
]

### Dataset Loading 

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the full CIFAR10 test set (10,000 samples)
full_testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform)

# Calculate split sizes
val_size = int(0.80 * len(full_testset))  # 7000
test_size = len(full_testset) - val_size  # 3000

# Randomly split into validation and test subsets
val_set, test_set = random_split(
    full_testset, [val_size, test_size],
    generator=torch.Generator().manual_seed(56)
)

# Create DataLoaders
valloader = DataLoader(val_set, batch_size=100, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=100, shuffle=False, num_workers=2)

print(f"Validation samples: {len(val_set)}")
print(f"Test samples: {len(test_set)}")

Validation samples: 8000
Test samples: 2000


## Base Models 

In [9]:
def visualize_test_and_roc(test_img, roc_imgs, local_labels, class_names=None):
    def denormalize(img_tensor, mean, std):
        mean = torch.tensor(mean).view(-1, 1, 1).to(img_tensor.device)
        std = torch.tensor(std).view(-1, 1, 1).to(img_tensor.device)
        return img_tensor * std + mean

    mean = [0.485, 0.456, 0.406]
    std  = [0.229, 0.224, 0.225] 

    k = roc_imgs.size(0)
    ncols = min(k, 5)
    nrows = 1 + (k + ncols - 1) // ncols  # one row for test image + RoC images
    
    plt.figure(figsize=(3 * ncols, 3 * nrows))
    
    # Plot test image
    plt.subplot(nrows, ncols, 1)
    denorm_img = denormalize(test_img, mean, std).clamp(0, 1)
    img_np = TF.to_pil_image(denorm_img.cpu())
    plt.imshow(img_np)
    plt.title("Test Image")
    plt.axis("off")
    
    # Plot RoC images
    for i in range(k):
        plt.subplot(nrows, ncols, i + 2)
        denorm_img = denormalize(roc_imgs[i], mean, std).clamp(0, 1)
        img_np = TF.to_pil_image(denorm_img.cpu())
        label = local_labels[i]
        if class_names:
            label = class_names[label]
        plt.imshow(img_np)
        plt.title(f"RoC #{i+1}\nLabel: {label}")
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [10]:
def get_last_linear_layer(model):
    """
    Try to find the last Linear layer in the model,
    using common attribute names and fallback to scanning modules.
    """
    name = model.__class__.__name__.lower()

    # Common last layer attribute names to try (ViT, EfficientNet, etc)
    candidate_attrs = ['head', 'heads', 'classifier', 'fc', 'mlp_head']

    for attr in candidate_attrs:
        if hasattr(model, attr):
            layer = getattr(model, attr)
            # If it's directly a Linear layer
            if isinstance(layer, nn.Linear):
                return layer
            # If Sequential or Module, find last Linear inside it
            if isinstance(layer, nn.Sequential) or isinstance(layer, nn.Module):
                # Find last Linear inside this attribute recursively
                last_linear = None
                for child in reversed(list(layer.modules())):
                    if isinstance(child, nn.Linear):
                        last_linear = child
                        break
                if last_linear is not None:
                    return last_linear

    # Fallback: scan all modules and pick the last Linear
    last_linear = None
    for m in model.modules():
        if isinstance(m, nn.Linear):
            last_linear = m
    if last_linear is not None:
        return last_linear

    raise RuntimeError("No Linear layer found in model")


def get_features_before_last_linear(model, x):
    features = {}

    def find_last_linear(module):
        last_linear = None
        for m in module.modules():
            if isinstance(m, torch.nn.Linear):
                last_linear = m
        return last_linear

    last_linear = find_last_linear(model)
    if last_linear is None:
        raise RuntimeError("No Linear layer found in model")

    def hook(module, input, output):
        features['feat'] = input[0].detach()

    handle = last_linear.register_forward_hook(hook)

    model.eval()
    with torch.no_grad():
        _ = model(x)

    handle.remove()

    if 'feat' not in features:
        raise RuntimeError("Failed to capture features from last linear layer")

    return features['feat']

In [11]:
def fire_check(local_labels, preds, per_class_min=1):
    local_labels = np.asarray(local_labels)

    # If preds are logits/probs, convert to labels
    preds = np.asarray(preds)
    if preds.ndim > 1:
        preds = preds.argmax(axis=1)

    # Classes present in the RoC (unique, not repeated)
    classes_in_roc = np.unique(local_labels)

    # Check: for each class c in RoC, there is at least `per_class_min` correct prediction
    missing = []
    for c in classes_in_roc:
        mask = (local_labels == c)
        n_correct = int(np.sum(preds[mask] == c))
        if n_correct < per_class_min:
            missing.append((int(c), n_correct))  # track which class is short

    fire_ok = (len(missing) == 0)
    return fire_ok

## VisionDES 

In [12]:

class VisionDES: 
    def __init__(self, dsel_dataset, pool): 
        self.dsel_dataset = dsel_dataset
        self.dsel_loader = DataLoader(dsel_dataset, batch_size=32, shuffle=False) 
        self.dino_model = timm.create_model('vit_base_patch8_224.dino', pretrained=True).to(device)
        self.dino_model.eval()  
        self.pool = pool 

        self.suspected_model_votes = [] 
        
        
    def dino_embedder(self, images):
        if images.shape[1] == 1:
            images = images.repeat(1, 3, 1, 1)

        images = F.interpolate(images, size=(224, 224), mode="bilinear", align_corners=False) 
        return self.dino_model.forward_features(images)


    def fit(self): 
        dsel_embeddings = []
        dsel_labels = []
    
        with torch.no_grad():
            for imgs, labels in tqdm(self.dsel_loader):
                imgs = imgs.to(device)
                embs = self.dino_embedder(imgs).cpu()  
                dsel_embeddings.append(embs)
                dsel_labels.append(labels)
    
        # Keep as tensor
        dsel_embeddings_tensor = torch.cat(dsel_embeddings).detach().cpu()  
        cls_tensor = dsel_embeddings_tensor[:, 0, :]  
    
        # Convert to NumPy
        cls_embeddings = np.ascontiguousarray(cls_tensor.numpy(), dtype='float32')
        self.dsel_embeddings = cls_embeddings
        self.dsel_labels = torch.cat(dsel_labels).numpy()
    
        # Build FAISS index
        embedding_dim = cls_embeddings.shape[1]
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.index.add(cls_embeddings)

    
    def get_output_size(self, model):
        """
        Returns the output size (number of classes) from various model architectures.
        """
        if hasattr(model, 'fc'):
            return model.fc.out_features
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Sequential):
                return model.classifier[-1].out_features
            else:
                return model.classifier.out_features
        elif hasattr(model, 'heads'):  # ViT / DINO from torchvision
            return model.heads.head.out_features
        elif hasattr(model, 'head'):  # ViT/Swin from timm
            return model.head.out_features
        else:
            raise AttributeError("Cannot determine output size of the model.")


    def predict_weighted_robust(self, test_img, k=7, return_logits=False, explain=False, top=False, n=3, use_fire=False, per_class_min=1, 
                                use_sim=False, sim_threshold=0, alpha=0.6, knorae=False):
        # Step 1: Get DINO CLS embedding for the test image
        img_for_dino = test_img.unsqueeze(0).to(device)
        img_for_dino = F.interpolate(img_for_dino, size=(224, 224), mode="bilinear", align_corners=False) 
        
        with torch.no_grad():
            test_emb = self.dino_model.forward_features(img_for_dino).cpu().numpy().astype('float32')
            test_emb = test_emb[:, 0, :]  # CLS token only
    
        # Step 2: Find k nearest neighbors in FAISS (Region of Competence)
        distances, neighbors = self.index.search(test_emb, k)
        neighbor_idxs = neighbors[0]
        local_labels = self.dsel_labels[neighbor_idxs]
        local_labels = np.array(local_labels).flatten()
    
        # Step 3: Get RoC images
        with torch.no_grad():
            roc_imgs = torch.stack([self.dsel_dataset[idx][0] for idx in neighbor_idxs]).to(device)
    
        # Step 4: Evaluate classifiers — compute competence and feature similarity
        competences, soft_outputs, feature_similarities, passed_fire, correct_counts = [], [], [], [], []
    
        test_img_batch = test_img.unsqueeze(0).to(device)
    
        for clf in self.pool:
            clf.eval()
            with torch.no_grad():
                outputs = clf(roc_imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()
                correct = (preds == local_labels).sum()
                competence = correct / k
                competences.append(competence)
                correct_counts.append(correct)

                # 🔥 FIRE check: at least one correct per class
                fire_ok = fire_check(local_labels, preds, per_class_min=per_class_min) 
                passed_fire.append(fire_ok)

                logits = clf(test_img.unsqueeze(0).to(device)).squeeze(0)
                probs = softmax(logits, dim=0)
                soft_outputs.append(probs)

                # Feature similarity using ResNet embeddings
                test_feat = get_features_before_last_linear(clf, test_img.unsqueeze(0).to(device))
                roc_feats = get_features_before_last_linear(clf, roc_imgs)
                mean_feat = roc_feats.mean(dim=0, keepdim=True)

                sim = cosine_similarity(test_feat / test_feat.norm(), mean_feat / mean_feat.norm(), dim=1)
                feature_similarities.append(sim.item())

        # 5️⃣ KNORA-E selection logic
        if knorae: 
            selected_indices = []
            required_correct = k  # start with strict condition (all correct)
            while required_correct >= 1 and not selected_indices:
                selected_indices = [i for i, c in enumerate(correct_counts) if c >= required_correct]
                required_correct -= 1
    
            if not selected_indices:  # failsafe: fall back to all models
                selected_indices = list(range(len(self.pool)))

            for i in range(len(self.pool)): 
                if i not in selected_indices: 
                    competences[i] = 0.0 
                    # feature_similarities[i] = 0.0 
    
        # Step 5: Combine competence & feature similarity into a score
        if use_sim:
            selected_feature_sims = [s if s > sim_threshold else 0.0 for s in feature_similarities]
            combined_scores = [alpha * c + (1 - alpha) * s for c, s in zip(competences, selected_feature_sims)]
        else:
            combined_scores = competences[:]
        
        if use_fire:
            combined_scores = [s if passed_fire[i] else 0.0 for i, s in enumerate(combined_scores)]
        
        # Step 6: Select models
        if top:
            top_n_idx = np.argsort(combined_scores)[::-1][:n]  # top-n in descending order
            total_score = sum(combined_scores[i] for i in top_n_idx)
            if total_score == 0:
                weights = [1.0 / n] * n
            else:
                weights = [combined_scores[i] / total_score for i in top_n_idx]
        else:
            total_score = sum(combined_scores)
            if total_score == 0:
                weights = [1.0 / len(self.pool)] * len(self.pool)
            else:
                weights = [s / total_score for s in combined_scores]
    
        # Step 7: Weighted aggregation of top-n classifier outputs
        num_classes = 100
        weighted_logits = torch.zeros(100).to(device)

        if top: 
            for idx, weight in zip(top_n_idx, weights):
                # print(idx, weight, soft_outputs[idx][:10])
                weighted_logits += weight * soft_outputs[idx]
        else: 
            for prob, weight in zip(soft_outputs, weights):
                weighted_logits += weight * prob
            
    
        # Step 8: Keep track of suspected attacked model
        min_sim_idx = int(np.argmin(feature_similarities))
        self.suspected_model_votes.append(min_sim_idx)

        # Step 8: Optional explainability
        # Step 8: Optional explainability
        if explain:
            print("\nExplainability Report:")
        
            if top:  # only report top-n models
                for idx, weight in zip(top_n_idx, weights):
                    prob = soft_outputs[idx]
                    comp = competences[idx]
                    sim = feature_similarities[idx]
                    fire = passed_fire[idx]
                    com_score = combined_scores[idx]
        
                    pred_class = prob.argmax().item()
                    conf = prob[pred_class].item()
                    topk = torch.topk(prob, k=5)
        
                    print(f"Model #{idx}: {self.pool[idx].__class__.__name__}")
                    print(f"  - Competence: {comp:.4f}")
                    print(f"  - Feature similarity: {sim:.4f}")
                    print(f"  - Combined score: {com_score:.4f}")
                    print(f"  - Combined weight: {weight:.4f}")
                    print(f"  - 🔥 FIRE: {fire}")
                    print(f"  - Predicted class: {pred_class} with confidence {conf:.4f}")
                    print(f"  - Top-5: {topk.indices.tolist()} → {[round(p.item(), 3) for p in topk.values]}")
                    print("-" * 50)
            else:  # report all models
                for idx, (comp, sim, weight, prob, fire, com_score) in enumerate(
                    zip(competences, feature_similarities, weights, soft_outputs, passed_fire, combined_scores)
                ):
                    pred_class = prob.argmax().item()
                    conf = prob[pred_class].item()
                    topk = torch.topk(prob, k=5)
        
                    print(f"Model #{idx}: {self.pool[idx].__class__.__name__}")
                    print(f"  - Competence: {comp:.4f}")
                    print(f"  - Feature similarity: {sim:.4f}")
                    print(f"  - Combined score: {com_score:.4f}")
                    print(f"  - Combined weight: {weight:.4f}")
                    print(f"  - 🔥 FIRE: {fire}")
                    print(f"  - Predicted class: {pred_class} with confidence {conf:.4f}")
                    print(f"  - Top-5: {topk.indices.tolist()} → {[round(p.item(), 3) for p in topk.values]}")
                    print("-" * 50)
        
            print(f"\n🧠 Final prediction: {weighted_logits.argmax().item()}")
            top5 = torch.topk(weighted_logits, k=5)
            print(f"🔝 Top-5 predictions:")
            for i in range(5):
                print(f"  - Class {top5.indices[i].item()}: {top5.values[i].item():.4f}")
        
            print("\nModel weight distribution:")
            if top:
                for idx, w in zip(top_n_idx, weights):
                    print(f"  Model #{idx}: {w:.4f}")
            else:
                for idx, w in enumerate(weights):
                    print(f"  Model #{idx}: {w:.4f}")
        
            print(f"Suspected attacked model: Model #{min_sim_idx} ({self.pool[min_sim_idx].__class__.__name__})")
        
            print("\nRoC visualization:")
            visualize_test_and_roc(test_img.squeeze(0), roc_imgs, local_labels)
            print(distances)

        if return_logits:
            return weighted_logits
        return weighted_logits.argmax().item()

    def predict(self, dataloader): 
        total = 0
        correct = 0 

        dataset_offset = 0 


### Evaluate DES 

In [13]:
from sklearn.metrics import precision_recall_fscore_support

def evaluate_des_with_fails(des_model, dataloader, dataset, average="weighted"):
    total = 0
    correct = 0
    failed_indices = []
    dataset_offset = 0  # to map dataloader batches back to dataset indices
    
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in tqdm(dataloader, desc="Testing VisionDES"):
            imgs, labels = imgs.to(device), labels.to(device)

            preds = []
            for idx_in_batch, img in enumerate(imgs):
                pred = des_model.predict_weighted_robust(
                    img, k=5, return_logits=False, explain=False, top=False, n=3, 
                    use_fire=False, per_class_min=1, use_sim=True, sim_threshold=0, 
                    alpha=0.6, knorae=True
                )
                preds.append(pred)

                # Collect failures (map batch index to dataset index)
                if pred != labels[idx_in_batch].item():
                    failed_indices.append(dataset_offset + idx_in_batch)

            preds = torch.tensor(preds).to(device)

            # accumulate results
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            dataset_offset += labels.size(0)

    acc = correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average=average, zero_division=0
    )
    
    print(f"✅ Accuracy: {acc*100:.2f} | Precision: {precision*100:.2f} | Recall: {recall*100:.2f} | F1: {f1*100:.2f}")
    
    return failed_indices


In [14]:
des_model = VisionDES(val_set, trained_pool)
des_model.fit()

100%|█████████████████████████████████████████████████████████████████| 250/250 [02:37<00:00,  1.58it/s]


## Static Ensemble 

In [15]:
class SoftVotingEnsemble:
    def __init__(self, models, device='cpu'):
        self.models = models
        self.device = device
        for model in self.models:
            model.eval().to(device)

    def predict(self, images, return_probs=False):
        probs = []
        with torch.no_grad():
            for model in self.models:
                outputs = model(images.to(self.device))
                softmaxed = torch.softmax(outputs, dim=1)
                probs.append(softmaxed.cpu().numpy())
        mean_probs = np.mean(np.stack(probs), axis=0)
        if return_probs:
            return mean_probs
        return np.argmax(mean_probs, axis=1)

    def predict_single_with_probs(self, image):
        image = image.unsqueeze(0)  # Shape [1, C, H, W]
        with torch.no_grad():
            model_probs = []
            for model in self.models:
                logits = model(image.to(self.device))
                softmaxed = torch.softmax(logits, dim=1)
                model_probs.append(softmaxed.cpu().numpy())
            mean_probs = np.mean(np.stack(model_probs), axis=0)
            probs = mean_probs[0]
            pred = np.argmax(probs)
            return probs, pred

In [16]:
import torch
import numpy as np
from collections import Counter

class HardVotingEnsemble:
    def __init__(self, models, device='cpu'):
        self.models = models
        self.device = device
        for model in self.models:
            model.eval().to(device)

    def predict(self, images):
        all_preds = []
        with torch.no_grad():
            for model in self.models:
                outputs = model(images.to(self.device))
                preds = torch.argmax(outputs, dim=1)
                all_preds.append(preds.cpu().numpy())

        # shape: [n_models, batch_size] → transpose to [batch_size, n_models]
        all_preds = np.stack(all_preds, axis=0).T  

        # majority vote for each sample
        final_preds = []
        for preds in all_preds:
            most_common = Counter(preds).most_common(1)[0][0]
            final_preds.append(most_common)

        return np.array(final_preds)

    def predict_single_with_probs(self, image):
        image = image.unsqueeze(0)  # Shape [1, C, H, W]
        votes = []
        with torch.no_grad():
            for model in self.models:
                logits = model(image.to(self.device))
                pred = torch.argmax(logits, dim=1).item()
                votes.append(pred)

        # majority voting
        final_pred = Counter(votes).most_common(1)[0][0]
        return votes, final_pred


### Attack 

In [17]:
from torch.utils.data import TensorDataset, DataLoader
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from collections import defaultdict


def predict_for_ensembles(ensemble_model, test_loader, ens_type): 
    metrics = {
        'accuracy': [],
        'f1': [],
        'auc': []
    } 
    y_true, y_pred, y_prob = [], [], [] 

    for imgs, labels in tqdm(test_loader):
        img = imgs[0]
        label = labels.item()

        if ens_type == "soft": 
            probs, pred = ensemble_model.predict_single_with_probs(img)
        if ens_type == "des+": 
            logits = ensemble_model.predict_weighted_robust(
                    img, k=10, return_logits=True, explain=False, top=True, n=3, 
                    use_fire=False, per_class_min=1, use_sim=True, sim_threshold=0.4, 
                    alpha=0.5, knorae=False
                )
            probs = torch.softmax(logits, dim=0).cpu().numpy()
            pred = np.argmax(probs)
    
        y_true.append(label)
        y_pred.append(pred)
        y_prob.append(probs)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred, average='macro') * 100
    try:
        auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
    except:
        auc = float('nan')

    print(f"{ens_type} Accuracy: {acc:.2f}%  | F1: {f1:.2f}%  | AUC: {auc:.2f}%")

    metrics['accuracy'].append(acc)
    metrics['f1'].append(f1)
    metrics['auc'].append(auc) 
    # print(y_prob)

    return metrics

In [18]:
def evaluate_classifiers(pool, test_dataset, n_classes):
    loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    results = []

    for i, model in enumerate(pool):
        model = model.to(device)
        model.eval()
        y_true, y_pred, y_prob = [], [], []

        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device), labels.squeeze().long().to(device)
                outputs = model(imgs)
                preds = outputs.argmax(dim=1)
                probs = torch.softmax(outputs, dim=1)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
                y_prob.extend(probs.cpu().numpy())

        acc = (np.array(y_true) == np.array(y_pred)).mean() * 100
        f1 = f1_score(y_true, y_pred, average='macro') * 100
        try:
            auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
        except ValueError:
            auc = float('nan')  # Handle single-class test sets

        print(f"{model.__class__.__name__}: {acc}")

        results.append({
            'classifier': model.__class__.__name__,
            'accuracy': acc,
            'f1': f1,
            'auc': auc,
        })

    return results

In [19]:
soft_ensemble = SoftVotingEnsemble(trained_pool, device)
hard_ensemble = HardVotingEnsemble(trained_pool, device)

### Multi 

In [20]:
x_test = []
y_test = []
    
for img, label in test_set:
    x_test.append(img.numpy())  # if you want channels_last (HWC)
    y_test.append(label)
        
x_test = np.array(x_test)
y_test = np.array(y_test)

In [21]:
EPSILON = 0.02
EPSILON = 8/255 
EPSILON = 0.05 

In [22]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0] 
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True )

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv = pgd_attack.generate(x=x_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
adv_tensor = torch.tensor(x_test_adv).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset = TensorDataset(adv_tensor, labels_tensor)

adv_test_loader = DataLoader(adv_dataset, batch_size=1, shuffle=False) 

In [24]:
_ = evaluate_classifiers(trained_pool, adv_dataset, 100)

VGG: 12.1
ResNet: 62.74999999999999
DenseNet: 62.150000000000006
GoogleNet: 63.5
Xception: 63.1
InceptionV3: 65.8


In [25]:
predict_for_ensembles(soft_ensemble, adv_test_loader, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader, "soft")
predict_for_ensembles(des_model, adv_test_loader, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:15<00:00, 14.72it/s]


soft Accuracy: 66.00%  | F1: 65.38%  | AUC: 98.97%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:10<00:00, 15.30it/s]


soft Accuracy: 64.95%  | F1: 64.46%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:20<00:00,  3.57it/s]

des+ Accuracy: 70.60%  | F1: 70.25%  | AUC: 98.58%


{'accuracy': [70.6],
 'f1': [70.24915516515343],
 'auc': [np.float64(98.57989078754198)]}

In [26]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0, 4] 
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True )

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv_1 = pgd_attack.generate(x=x_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
adv_tensor_1 = torch.tensor(x_test_adv_1).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset_1 = TensorDataset(adv_tensor_1, labels_tensor)

adv_test_loader_1 = DataLoader(adv_dataset_1, batch_size=1, shuffle=False) 

In [28]:
_ = evaluate_classifiers(trained_pool, adv_dataset_1, 100)

VGG: 31.2
ResNet: 51.349999999999994
DenseNet: 51.300000000000004
GoogleNet: 51.849999999999994
Xception: 13.3
InceptionV3: 53.949999999999996


In [29]:
predict_for_ensembles(soft_ensemble, adv_test_loader_1, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader_1, "soft")
predict_for_ensembles(des_model, adv_test_loader_1, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:10<00:00, 15.36it/s]


soft Accuracy: 46.90%  | F1: 46.59%  | AUC: 97.51%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:16<00:00, 14.70it/s]


soft Accuracy: 46.75%  | F1: 46.47%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:16<00:00,  3.60it/s]

des+ Accuracy: 58.55%  | F1: 58.47%  | AUC: 95.83%


{'accuracy': [58.550000000000004],
 'f1': [58.47404310597777],
 'auc': [np.float64(95.82809166075324)]}

In [30]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0, 3, 4] 
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True )

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv_2 = pgd_attack.generate(x=x_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
adv_tensor_2 = torch.tensor(x_test_adv_2).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset_2 = TensorDataset(adv_tensor_2, labels_tensor)

adv_test_loader_2 = DataLoader(adv_dataset_2, batch_size=1, shuffle=False) 

In [32]:
_ = evaluate_classifiers(trained_pool, adv_dataset_2, 1000) 

VGG: 30.8
ResNet: 43.65
DenseNet: 42.199999999999996
GoogleNet: 17.0
Xception: 20.0
InceptionV3: 47.85


In [33]:
predict_for_ensembles(soft_ensemble, adv_test_loader_2, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader_2, "soft")
predict_for_ensembles(des_model, adv_test_loader_2, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:14<00:00, 14.90it/s]


soft Accuracy: 36.60%  | F1: 36.32%  | AUC: 95.40%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:22<00:00, 14.02it/s]


soft Accuracy: 37.90%  | F1: 37.72%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:37<00:00,  3.46it/s]

des+ Accuracy: 47.15%  | F1: 47.03%  | AUC: 91.74%


{'accuracy': [47.15],
 'f1': [47.033533005775816],
 'auc': [np.float64(91.74367598011438)]}

In [34]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0, 1, 3, 4] 
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True)

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv_3 = pgd_attack.generate(x=x_test)


adv_tensor_3 = torch.tensor(x_test_adv_3).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset_3 = TensorDataset(adv_tensor_3, labels_tensor)

adv_test_loader_3 = DataLoader(adv_dataset_3, batch_size=1, shuffle=False) 

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [35]:
_ = evaluate_classifiers(trained_pool, adv_dataset_3, 1000) 

VGG: 28.050000000000004
ResNet: 20.599999999999998
DenseNet: 31.45
GoogleNet: 17.2
Xception: 20.150000000000002
InceptionV3: 39.550000000000004


In [36]:
predict_for_ensembles(soft_ensemble, adv_test_loader_3, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader_3, "soft")
predict_for_ensembles(des_model, adv_test_loader_3, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:04<00:00, 16.09it/s]


soft Accuracy: 27.45%  | F1: 27.47%  | AUC: 90.62%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:05<00:00, 15.93it/s]


soft Accuracy: 28.30%  | F1: 28.30%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:23<00:00,  3.55it/s]

des+ Accuracy: 35.50%  | F1: 35.45%  | AUC: 83.55%


{'accuracy': [35.5],
 'f1': [35.45091207956732],
 'auc': [np.float64(83.54818442941038)]}

In [37]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0, 1, 3, 4, 5]  
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True )

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv_4 = pgd_attack.generate(x=x_test)


adv_tensor_4 = torch.tensor(x_test_adv_4).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset_4 = TensorDataset(adv_tensor_4, labels_tensor)

adv_test_loader_4 = DataLoader(adv_dataset_4, batch_size=1, shuffle=False) 

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [38]:
_ = evaluate_classifiers(trained_pool, adv_dataset_4, 1000) 

VGG: 27.05
ResNet: 19.2
DenseNet: 28.999999999999996
GoogleNet: 16.7
Xception: 19.6
InceptionV3: 23.1


In [39]:
predict_for_ensembles(soft_ensemble, adv_test_loader_4, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader_4, "soft")
predict_for_ensembles(des_model, adv_test_loader_4, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:15<00:00, 14.80it/s]


soft Accuracy: 23.75%  | F1: 23.84%  | AUC: 85.76%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:01<00:00, 16.45it/s]


soft Accuracy: 24.60%  | F1: 24.70%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:19<00:00,  3.58it/s]

des+ Accuracy: 29.80%  | F1: 29.90%  | AUC: 76.47%


{'accuracy': [29.799999999999997],
 'f1': [29.90099715332521],
 'auc': [np.float64(76.46948199400538)]}

In [40]:
from art.estimators.classification import EnsembleClassifier
from art.attacks.evasion import ProjectedGradientDescent

selected_pool_indices = [0, 1, 2, 3, 4, 5] 
# Wrap your pool into ART classifiers
art_classifiers = []
for idx in selected_pool_indices:
    model = trained_pool[idx].to(device).eval()
    dummy_optimizer = torch.optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    art_clf = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(3, 32, 32),  # <-- channels_first shape
        nb_classes=100,
        clip_values=(0.0, 1.0),
        channels_first=True,        # <-- make it explicit
    )
    art_classifiers.append(art_clf)

# Create ensemble wrapper
ensemble_clf = EnsembleClassifier(classifiers=art_classifiers, channels_first=True )

# Run PGD on the whole ensemble
pgd_attack = ProjectedGradientDescent(estimator=ensemble_clf, eps=EPSILON, eps_step=0.01, max_iter=100)
x_test_adv_5 = pgd_attack.generate(x=x_test)


adv_tensor_5 = torch.tensor(x_test_adv_5).float()
labels_tensor = torch.tensor(y_test).long()
adv_dataset_5 = TensorDataset(adv_tensor_5, labels_tensor)

adv_test_loader_5 = DataLoader(adv_dataset_5, batch_size=1, shuffle=False) 

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [41]:
_ = evaluate_classifiers(trained_pool, adv_dataset_5, 1000) 

VGG: 25.85
ResNet: 18.099999999999998
DenseNet: 19.0
GoogleNet: 15.65
Xception: 20.1
InceptionV3: 21.55


In [ ]:
predict_for_ensembles(soft_ensemble, adv_test_loader_5, "soft")
predict_for_ensembles(hard_ensemble, adv_test_loader_5, "soft")
predict_for_ensembles(des_model, adv_test_loader_5, "des+")

100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:04<00:00, 16.06it/s]


soft Accuracy: 20.70%  | F1: 21.14%  | AUC: 81.65%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [02:08<00:00, 15.51it/s]


soft Accuracy: 21.85%  | F1: 22.25%  | AUC: nan%


100%|███████████████████████████████████████████████████████████████| 2000/2000 [09:12<00:00,  3.62it/s]

des+ Accuracy: 27.45%  | F1: 27.73%  | AUC: 71.69%


{'accuracy': [27.450000000000003],
 'f1': [27.72994119842178],
 'auc': [np.float64(71.68637422019093)]}